# Generating puzzles with multiple solutions
Puzzles with multiple solutions can create very interesting effects, such as the puzzle shown in the video "[How can a jigsaw have two distinct solutions?](https://www.youtube.com/watch?v=b5nElEbbnfU)" on the Stand-Up Maths Youtube channel where the two ways of solving the jigsaw puzzle either create an image of a coffee cup or a donut. This page describes how such puzzles can be generated by first picking a random scrambling of the puzzle pieces, and then generating a puzzle that is solvable in both its unscrambled and scrambled state. There is also Python code.

In [38]:
# First, import the Python libraries we need
from typing import Dict, Tuple, Set, List
import numpy as np
import random
from ipycanvas import Canvas
from ipywidgets import Image as WidgetImage
from PIL import Image
import math

# Describing puzzles using vectors
In order for a jigsaw puzzle to be solvable in more than one way, there must be several connections between pieces that have the same shape. Otherwise, if every connection was unique, each side of a piece would only be able to connect with a single side of a single other piece, and there would only be a single solution to the puzzle. This means that each side of each puzzle piece can be grouped with other puzzle piece sides with a similar shape, and these groups can be given numbers.

It is useful to give the shapes numbers so that shapes that are compatible with each other have inverse numbers. For example, shape 1 is compatible with shape -1, shape 2 is compatible with shape -2, and so forth. This makes it easy to check whether two shapes are compatible: they are compatible if they add up to 0, and incompatible if they do not. I prefer to think of the positive numbers as all the shapes that stick out and the negative numbers as the indented shapes, but this is fully arbitrary. Finally, the outer sides of the pieces at the edge of the jigsaw puzzle are just shaped like straight lines, and do not connect to anything. These non-connecting shapes are labelled 0.

A puzzle can be described mathematically as a vector containing the shapes of each side of all the pieces of the puzzle. The order of the sides is also arbitrary, but here we order the sides in order left, top, right, bottom, starting with the top left piece and going row by row. The ordering of sides for a 2x2 puzzle is illustrated in the following figure.

<img src="sidenumbers.png" style="width:500px; margin-left:auto; margin-right:auto;"/>

Thus, if we assume that all the tabs in the image are in shape group 1 and all the indentations are in shape group -1, the vector describing this puzzle is:
$$
    \vec{p} = \begin{pmatrix}
        0 \\
        0 \\
        -1 \\
        1 \\
        1 \\
        0 \\
        0 \\
        1 \\
        0 \\
        -1 \\
        -1 \\
        0 \\
        1 \\
        -1 \\
        0 \\
        0
    \end{pmatrix}
$$

In order to solve a puzzle, the pieces have to be moved around and rotated. This can be done by performing matrix operations on the vector. For instance, the matrix that swaps the position of the top left and bottom right pieces looks like this, and then rotates the bottom right piece once counter-clockwise, looks like this:
$$
    \mathbf{T} = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    \end{pmatrix}
$$

# Validating if a puzzle is solved
A puzzle can be validated by checking each entry in the puzzle vector: if the entry corresponds to a side on the edge of the puzzle, its value must be zero. If an entry corresponds to a side which connects to another side in the puzzle, the sum of that entry and the other side's entry must be zero. This set of operations can be encoded into a verification matrix, $\mathbf{V}$. For example, the verification matrix for a 2x2 puzzle looks like this:
$$
    \mathbf{V} = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$
This matrix extracts only the entry itself for sides on the edge of the jigsaw puzzle, while it adds the value of both entries in the case of sides that face each other. Similar matrices can be constructed for any other dimension of jigsaw puzzle.

Using the verification matrix, puzzles can be validated by ensuring that they satisfy the following equation
$$
    \mathbf{V}\vec{p} = \vec{0}
$$

In [39]:
def generate_verification_matrix(width: int, height: int) -> np.array:
    number_of_sides = 4*width*height
    # The verification matrix always uses the value of the entry itself, so start with an identity matrix
    verification_matrix = np.identity(number_of_sides, np.int16)

    # Go through each puzzle piece and add its connection to the right and down
    for x in range(width):
        for y in range(height):
            piece_position = y*width + x
            # Don't add right connection if we're in the rightmost row
            if x != width - 1:
                other_piece_position = y*width + x + 1
                right_side_of_left_piece = 4*piece_position + 2
                left_side_of_right_piece = 4*other_piece_position
                verification_matrix[right_side_of_left_piece][left_side_of_right_piece] = 1
                verification_matrix[left_side_of_right_piece][right_side_of_left_piece] = 1
            # Also don't add down connection if we're in the bottom row
            if y != height - 1:
                other_piece_position = (y+1)*width + x
                bottom_side_of_top_piece = 4*piece_position + 3
                top_side_of_bottom_piece = 4*other_piece_position + 1
                verification_matrix[top_side_of_bottom_piece][bottom_side_of_top_piece] = 1
                verification_matrix[bottom_side_of_top_piece][top_side_of_bottom_piece] = 1

    return verification_matrix

 # Finding puzzles with multiple solutions
 Any arbitrary scrambling (i.e. movement and/or rotation of the pieces) of a puzzle can be described using a transformation matrix, $\mathbf{T}$. If the matrix equation $\mathbf{V}\mathbf{T}\vec{p} = \vec{0}$ holds, then this scrambling is a solution to the puzzle. For a given transformation $\mathbf{T}$ and a given verification matrix $\mathbf{V}$, it is possible to construct a puzzle vector $\vec{p}$ such that the following system of matrix equations hold true:
 $$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
 $$
 $$
    \mathbf{V}\vec{p} = \vec{0}
 $$

A puzzle $\vec{p}$ that satisfies these constraints must have at least two solutions, since both its unscrambled state and its state that is scrambled according to $\mathbf{T}$ are solutions.

In [40]:
def generate_solvable_puzzle(verification_matrix: np.array, transformation_matrix: np.array) -> np.array:
    # Both the verification matrix and the transformation matrix are square matrices, with a number of rows equal to the
    # number of sides in the puzzle
    trial_solution = np.zeros((len(verification_matrix), 1), np.int16)

    # Find each pair of sides that are constrained by each other in the unscrambled puzzle
    constraint_matrices = [verification_matrix, verification_matrix @ transformation_matrix]
    pairs = {}
    for constraint_matrix in constraint_matrices:
        for row in constraint_matrix:
            constrained_sides = []
            for index, value in enumerate(row):
                if value != 0:
                    constrained_sides.append(index)
            # If this side is not constrained by any other, it is an edge piece.
            # We'll ignore it and leave its value at 0
            if len(constrained_sides) != 2:
                continue

            for i in range(2):
                side = constrained_sides[i]
                other_side = constrained_sides[(i + 1) % 2]
                if side not in pairs:
                    pairs[side] = set()
                pairs[side].add(other_side)

    # Pick a side that does not have a set shape, set an arbitrary shape,
    # set all other sides that are constrained by this.
    # Repeat until no unset sides are left
    unset_sides = set(side for side in pairs)
    shape_number = 1

    while len(unset_sides) > 0:
        starting_side = unset_sides.pop()
        trial_solution[starting_side] = shape_number
        shape_number += 1

        affected_sides = [starting_side]
        affected_side_counter = 0
        while affected_side_counter < len(affected_sides):
            current_side = affected_sides[affected_side_counter]
            unset_sides.discard(current_side)
            for other_side in pairs[current_side]:
                trial_solution[other_side] = -trial_solution[current_side]
                if other_side not in affected_sides:
                    affected_sides.append(other_side)

            affected_side_counter += 1

    return trial_solution

# Random transformations and scramble similarity
Since we now know how to generate a puzzle that has at least two solutions given an arbitrary transformation, we can generate a random transformation and build a puzzle with multiple solutions using that. However, it is important to keep in mind that the transformation can't be completely arbitrary: only corner pieces can occupy the corners of the jigsaw puzzle, and only edge pieces can occupy the edges, and their orientation is determined by which edge they're on. All middle pieces can be arbitrarily rearranged and rotated.

In the video, Matt Parker also wants the two solutions to a puzzle to be as different as possible. One way of quantifying how similar the puzzle is after being scrambled is to count how many sides are adjacent to each other in the scrambled puzzle that were also adjacent in the original puzzle. This is easily countable: if every connecting shape in the original puzzle is unique, then every pair of sides that is still able to connect after scrambling the puzzle are sides that were connected in the original puzzle.

In [41]:
def categorize_position(width: int, height: int, x: int, y: int) -> Tuple[str, int]:
    if y == 0:
        if x == 0:
            return 'corner', 0
        elif x == width - 1:
            return 'corner', 3
        else:
            return 'edge', 0
    elif y == height - 1:
        if x == 0:
            return 'corner', 1
        elif x == width - 1:
            return 'corner', 2
        else:
            return 'edge', 2
    elif x == 0:
        return 'edge', 1
    elif x == width - 1:
        return 'edge', 3
    else:
        return 'middle', 0


def generate_random_transformation(width: int, height: int) -> np.array:
    number_of_sides = 4 * width * height
    transformation = np.zeros((number_of_sides, number_of_sides), np.int16)

    # Group pieces in corners, edges and middle, shuffle them, and then fill them in using the shuffled order
    corners = []
    edges = []
    middles = []
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            category, _ = categorize_position(width, height, x, y)
            if category == 'corner':
                corners.append(piece_number)
            elif category == 'edge':
                edges.append(piece_number)
            elif category == 'middle':
                middles.append(piece_number)

    random.shuffle(corners)
    random.shuffle(edges)
    random.shuffle(middles)

    # Now fill in the puzzle
    for transformed_x in range(width):
        for transformed_y in range(height):
            category, orientation = categorize_position(width, height, transformed_x, transformed_y)
            if category == 'corner':
                piece_number = corners.pop()
            elif category == 'edge':
                piece_number = edges.pop()
            else:
                piece_number = middles.pop()

            original_x = piece_number % width
            original_y = piece_number // width
            _, original_orientation = categorize_position(width, height, original_x, original_y)
            rotation = (orientation - original_orientation) % 4

            # If the piece is a middle piece, we can rotate it freely
            if category == 'middle':
                rotation = random.randrange(4)

            # Generate a rotation matrix, and place it in the correct position in the larger transformation matrix
            # Each rotation is counter-clockwise and 90 degrees. We can achieve this by shifting each row
            # in the identity matrix one up
            rotation_matrix = np.roll(np.identity(4, np.int16), -rotation, 0)

            # Place the rotation matrix in the correct spot in the larger transformation matrix
            new_piece_number = transformed_y * width + transformed_x
            transformation[
                4 * new_piece_number:4 * new_piece_number + 4,
                4 * piece_number:4 * piece_number + 4
            ] = rotation_matrix

    return transformation


def find_transformation_similarity(verification_matrix: np.array, transformation_matrix: np.array) -> int:
    # Generate a puzzle where each connection is unique
    unique_puzzle = np.zeros((len(verification_matrix), 1), np.int16)
    number_of_edges = 0
    shape_number = 1
    for row in verification_matrix:
        connected_sides = []
        for index, value in enumerate(row):
            if value != 0:
                connected_sides.append(index)
        if len(connected_sides) == 1:
            number_of_edges += 1
        else:
            unique_puzzle[connected_sides[0]] = shape_number
            unique_puzzle[connected_sides[1]] = -shape_number
            shape_number += 1

    # Now, the number of sides that are still touching in the transformed puzzle is equal to the number of zeros in the
    # vector generated by the verification matrix, minus the number of edges (which are always zero in the vector)
    verification_vector = verification_matrix @ transformation_matrix @ unique_puzzle
    similarity = -number_of_edges
    for entry in verification_vector:
        if entry == 0:
            similarity += 1
    # Each similarity is counted twice in the verification vector
    return similarity // 2

# Counting the number of solutions
In order to achieve the effect in the video where the puzzle motif changes from a coffee cup to a donut, the puzzle should only have two solutions, otherwise it would be possible to put it back together in different ways that create nonsensical motifs. Since the above method guarantees that a puzzle has *at least* two solutions, we need to count the number of possible solutions to ensure that the puzzle has *only* two solutions.

Counting the number of possible solutions amounts to finding the number of matrices $\mathbf{T}$ that satisfies the equation
$$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
$$
where $\mathbf{V}$ and $\vec{p}$ are known. I haven't been able to prove this, but experience shows that $\mathbf{V}$ is always non-invertible, so I don't think there exists a better way of finding $\mathbf{T}$ than trial and error.

Since counting the number of solutions takes a long time, it is useful to weed out puzzles that obviously have more than two solutions. Any puzzle with at least two solutions where two pieces have the exact same sides must have more solutions, since we can also solve the puzzle by just swapping those two pieces).

In [42]:
def has_duplicate_pieces(puzzle: np.array) -> bool:
    pieces = []
    for i in range(0, len(puzzle), 4):
        piece = (puzzle[i], puzzle[i + 1], puzzle[i + 2], puzzle[i + 3])
        rotations = [(puzzle[i + j % 4], puzzle[i + (j + 1) % 4], puzzle[i + (j + 2) % 4], puzzle[i + (j + 3) % 4]) for
                     j in range(4)]
        for other_piece in pieces:
            if other_piece in rotations:
                return True
        pieces.append(piece)
    return False


class PuzzlePiece:
    def __init__(self, piece_number: int, connection_shapes: List[int]):
        self.piece_number = piece_number
        self.connection_shapes = connection_shapes

    def get_side_shape(self, side_number: int, rotation_number: int):
        return self.connection_shapes[(side_number - rotation_number) % 4]


class PuzzleSolutionBuilder:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.placed_pieces = {}
        self.unfilled_neighbours = {}

    def place_piece(self, piece: PuzzlePiece, rotation: int, position_x: int, position_y: int) -> bool:
        neighbour_restrictions = []
        position = position_y * self.width + position_x

        # Check with neighbours that it fits
        for dx, dy, our_side_facing_neighbour, neighbour_side_facing_us in [(1, 0, 2, 0), (0, 1, 3, 1), (-1, 0, 0, 2),
                                                                            (0, -1, 1, 3)]:
            neighbour_x = position_x + dx
            neighbour_y = position_y + dy
            # Check we're still inside the board
            if (not 0 <= neighbour_x < self.width) or (not 0 <= neighbour_y < self.height):
                # If we're not, our side facing this way must have a shape of zero
                if piece.get_side_shape(our_side_facing_neighbour, rotation) != 0:
                    return False
                continue

            # Check if a neighbouring piece has been placed yet
            neighbour_position = neighbour_y * self.width + neighbour_x
            if neighbour_position in self.placed_pieces:
                # If there is a piece in the neighbouring location, check that this piece is compatible with that
                neighbour_piece, neighbour_rotation = self.placed_pieces[neighbour_position]
                if (
                        neighbour_piece.get_side_shape(neighbour_side_facing_us, neighbour_rotation)
                        + piece.get_side_shape(our_side_facing_neighbour, rotation)
                        != 0
                ):
                    # If we get here, the piece did not fit
                    return False
            else:
                # If there is no neighbour on this side, we should store what shape that neighbour needs to fit
                neighbour_restrictions.append((
                    neighbour_position,
                    neighbour_side_facing_us,
                    -piece.get_side_shape(our_side_facing_neighbour, rotation)
                ))

        # If we went through all the sides, the piece fits. Place it
        self.placed_pieces[position] = (piece, rotation)

        # This is no longer an unfilled neighbour
        if position in self.unfilled_neighbours:
            del self.unfilled_neighbours[position]

        # Add any new unfilled neighbours we found
        for neighbour_position, neighbour_side_facing_us, required_shape in neighbour_restrictions:
            if neighbour_position not in self.unfilled_neighbours:
                self.unfilled_neighbours[neighbour_position] = []
            self.unfilled_neighbours[neighbour_position].append((neighbour_side_facing_us, required_shape))

        return True

    def remove_last_piece(self) -> PuzzlePiece:
        piece_position, (piece, rotation) = self.placed_pieces.popitem()

        requirements_for_replacement_piece = []

        piece_x = piece_position % self.width
        piece_y = piece_position // self.width
        # Update the unfilled neighbours
        for dx, dy, our_side_facing_neighbour, neighbour_side_facing_us in [(1, 0, 2, 0), (0, 1, 3, 1), (-1, 0, 0, 2),
                                                                            (0, -1, 1, 3)]:
            neighbour_x = piece_x + dx
            neighbour_y = piece_y + dy

            # If this side is off the board, we will not need to do anything about any requirements
            if (not 0 <= neighbour_x < self.width) or (not 0 <= neighbour_y < self.height):
                continue

            # If there is still a neighbouring piece on this side, we need to require whichever piece
            # replaces this piece to fit with that neighbour
            neighbour_position = neighbour_y * width + neighbour_x
            if neighbour_position in self.placed_pieces:
                requirements_for_replacement_piece.append((
                    our_side_facing_neighbour,
                    piece.get_side_shape(our_side_facing_neighbour, rotation)
                ))
                continue

            # If there is no neighbour on this side, there should be an unfilled neighbour with a set of requirements there
            # Remove our requirements for that neighbour, and if there are no requirements left, remove the unfilled neighbour
            # completely
            our_requirement = (neighbour_side_facing_us, -piece.get_side_shape(our_side_facing_neighbour, rotation))
            self.unfilled_neighbours[neighbour_position].remove(our_requirement)
            if len(self.unfilled_neighbours[neighbour_position]) == 0:
                del self.unfilled_neighbours[neighbour_position]

        # Leave the requirements for any replacement pieces
        self.unfilled_neighbours[piece_position] = requirements_for_replacement_piece
        return piece

    def get_unfilled_neighbours(self) -> Dict[int, Tuple[int, int]]:
        return self.unfilled_neighbours

    def get_number_of_pieces(self) -> int:
        return len(self.placed_pieces)

    def is_finished(self) -> bool:
        return len(self.placed_pieces) == width * height


def count_solutions(
        puzzle: np.array,
        width: int,
        height: int
) -> int:
    # Do this in the same way Matt Parker does in the video. Just try every combination of pieces until we find one that works
    # Keep a running tally of solutions until we are done
    number_of_solutions = 0

    # Create pieces based on the puzzle input
    pieces = []
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            side_shapes = [int(shape) for shape in puzzle[4 * piece_number:4 * piece_number + 4, 0]]
            pieces.append(PuzzlePiece(piece_number, side_shapes))

    # Create a map from a given shape to the pieces that have that shape
    shape_to_pieces_map = {}
    for piece in pieces:
        for side_number in range(4):
            shape = piece.get_side_shape(side_number, 0)
            if shape not in shape_to_pieces_map:
                shape_to_pieces_map[shape] = set()
            shape_to_pieces_map[shape].add((piece, side_number))

    solution_builder = PuzzleSolutionBuilder(width, height)

    # We want to do a depth first search of the puzzle
    # Store it every time we have to make a choice, so we can go back later and do the opposite choice
    available_pieces = set(pieces)
    # Each choice stores which piece to place, its x and y position, rotation, and how many pieces were already
    # laid when making the choice
    choices = [(pieces[0], 0, 0, 0, 0)]
    while len(choices) > 0:
        piece_to_place, position_x, position_y, rotation, pieces_placed_when_making_choice = choices.pop()
        # Since we're doing a depth first search, we only need to remove pieces to get back to the state we were
        # in when we made this choice
        while solution_builder.get_number_of_pieces() > pieces_placed_when_making_choice:
            removed_piece = solution_builder.remove_last_piece()
            # The piece we just removed is available to be placed again
            available_pieces.add(removed_piece)

        # Try to place the piece
        if not solution_builder.place_piece(piece_to_place, rotation, position_x, position_y):
            # If we couldn't place the piece, this is a dead end, and we must return
            continue

        # The piece we placed is no longer available
        available_pieces.remove(piece_to_place)

        # We might have finished building the puzzle now
        if solution_builder.is_finished():
            number_of_solutions += 1
            continue

        # Check what options we have for placing the next puzzle piece
        options = {}
        for neighbour_position in solution_builder.get_unfilled_neighbours():
            requirements = solution_builder.get_unfilled_neighbours()[neighbour_position]
            # To start with, I'll just find all the pieces that fit one of the requirements
            # Maybe we can do more requirement checking later
            required_side, required_shape = requirements[0]

            # Find all pieces with the required shape, and make sure we only consider pieces that are available
            possible_pieces = [(piece, side) for piece, side in shape_to_pieces_map[required_shape] if piece in available_pieces]

            # Find the possible rotations for each possible piece
            options[neighbour_position] = []
            for possible_piece, side_number in possible_pieces:
                options[neighbour_position].append((possible_piece, required_side - side_number))

        # Place a piece in the position with the fewest options
        best_position = min(options, key=lambda position: len(options[position]))
        best_position_x = best_position % width
        best_position_y = best_position // width
        # Create a choice for each possible piece
        for piece, rotation in options[best_position]:
            choices.append((piece, best_position_x, best_position_y, rotation, solution_builder.get_number_of_pieces()))
    return number_of_solutions

The following cell contains code to draw the puzzle

In [43]:
# Define a function to draw the results
color_stops = [
    (0, 'white'),
    (0.2, 'black'),
    (0.4, 'yellow'),
    (0.6, 'green'),
    (0.8, 'blue'),
    (1.0, 'purple')
]

def draw_puzzle(puzzle, width, height, transformation=None):
    if transformation is None:
        canvas = Canvas(width=100*width, height=100*height, sync_image_data=True)
    else:
        canvas = Canvas(width=200*width + 50, height=100*height, sync_image_data=True)

    image = Image.open('mattandsteve.png')

    # Crop depending on puzzle aspect ratio
    desired_width_pixels = 100*width
    desired_height_pixels = 100*height

    width_resize = desired_width_pixels / image.width
    height_resize = desired_height_pixels / image.height

    if height_resize > width_resize:
        image = image.resize((int(height_resize*image.width), int(height_resize*image.height)), Image.LANCZOS)
        
        left_crop = 110 * height_resize
        if image.width - left_crop < 100*width:
            left_crop = image.width - 100*width
        image = image.crop((int(left_crop), 0, int(left_crop)+100*width, 100*height))
    else:
        image = image.resize((int(width_resize*image.width), int(width_resize*image.height)), Image.LANCZOS)

        top_crop = 100 * width_resize
        if image.height - top_crop < 100*height:
            top_crop = image.height - 100*height
        image = image.crop((0, int(top_crop), 100*width, int(top_crop) + 100*height))
    
    # Cut image into pieces
    image_pieces = {}
    for x in range(width):
        for y in range(height):
            piece_number = y*width + x
            image_piece = image.crop((x*100, y*100, (x+1)*100, (y+1)*100))
            image_pieces[piece_number] = image_piece

    # Draw original puzzle
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            image_piece = image_pieces[piece_number]
            canvas.put_image_data(np.asarray(image_piece), x*100, y*100)

    # Draw borders between puzzle pieces
    canvas.stroke_style = 'black'
    for x in range(width+1):
        canvas.stroke_line(x*100, 0, x*100, 100*height)
    for y in range(height+1):
        canvas.stroke_line(0, y*100, width*100, y*100)

    # Draw piece number at the centre of each piece
    canvas.stroke_style = 'white'
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            center_x = 100*x + 50
            center_y = 100*y + 50
            canvas.stroke_text(str(piece_number), center_x, center_y)

    # Draw shape type at the edge of each piece
    canvas.stroke_style = 'red'
    directions = {
        0: (-42, 0),
        1: (0, -40),
        2: (42, 0),
        3: (0, 40)
    }
    
    for index, value in enumerate(puzzle[:,0]):
        if value == 0:
            continue
        piece_number = index // 4
        center_x = 100 * (piece_number % width) + 50
        center_y = 100 * (piece_number // width) + 50

        direction = index % 4
        dx, dy = directions[direction]

        canvas.stroke_text(str(value), center_x+dx, center_y + dy)

    # Draw a dot above each number, to show that pieces are in their original orientation
    canvas.fill_style = 'green'
    for x in range(width):
        for y in range(height):
            location_x = x * 100 + 50
            location_y = y * 100 + 35
            canvas.fill_circle(location_x, location_y, 3)

    # If there is no transformation, we are done now
    if transformation is None:
        return canvas

    # Else, map where each piece has gone to and whether it is rotated
    transform_map = {}
    for new_piece_number in range(width*height):
        column_set_to_one = min(index for index, value in enumerate(transformation[new_piece_number*4, :]) if value != 0)
        old_piece_number = column_set_to_one // 4
        piece_rotation = column_set_to_one % 4
        transform_map[new_piece_number] = (old_piece_number, piece_rotation)

    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            old_piece_number, rotation = transform_map[piece_number]
            image_piece = image_pieces[old_piece_number]
            image_piece = image_piece.rotate(90*rotation)
            canvas.put_image_data(np.asarray(image_piece), (width+x)*100 + 50, y*100)

    # Draw borders between puzzle pieces
    canvas.stroke_style = 'black'
    for x in range(width+1):
        canvas.stroke_line((x+width)*100+50, 0, (x+width)*100+50, 100*height)
    for y in range(height+1):
        canvas.stroke_line(100*width+50, y*100, 200*width+50, y*100)

    # Draw piece numbers of the moved pieces
    canvas.stroke_style = 'white'
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            old_piece_number, _ = transform_map[piece_number]

            center_x = 100*(width + x) + 100
            center_y = 100*y + 50

            canvas.stroke_text(str(old_piece_number), center_x, center_y)

    # Draw connection shape types on the moved pieces
    canvas.stroke_style = 'red'
    directions = {
        0: (-42, 0),
        1: (0, -40),
        2: (42, 0),
        3: (0, 40)
    }
    
    for index, value in enumerate((T@puzzle)[:,0]):
        if value == 0:
            continue
        piece_number = index // 4
        center_x = 100 * (width  + piece_number % width) + 100
        center_y = 100 * (piece_number // width) + 50

        direction = index % 4
        dx, dy = directions[direction]

        canvas.stroke_text(str(value), center_x+dx, center_y + dy)

    # Draw green dots to show how each piece has been rotated
    canvas.fill_style = 'green'
    rotations = {
        0: (0, -15),
        1: (-15, 0),
        2: (0, 15),
        3: (15, 0)
    }
    for x in range(width):
        for y in range(height):
            piece_number = y * width + x
            _, rotation = transform_map[piece_number]

            center_x = 100*(width + x) + 100
            center_y = 100*y + 50

            dx, dy = rotations[rotation]
            canvas.fill_circle(center_x + dx, center_y + dy, 3)
    
    
    return canvas

In [44]:
width = 5
height = 5

V = generate_verification_matrix(width, height)
puzzle_count = 0
while True:
    T = None
    T_similarity = 1
    while T_similarity > 0:
        T = generate_random_transformation(width, height)
        T_similarity = find_transformation_similarity(V, T)
    p = generate_solvable_puzzle(V, T)
    if has_duplicate_pieces(p):
        continue
    
    number = count_solutions(p, width, height)
    puzzle_count += 1
    print(f'Investigated {puzzle_count} puzzles. The most recent one had {number} solutions.    ', end='\r')
    if number == 2:
        break
print()
draw_puzzle(p, width, height, T)

Investigated 1 puzzles. The most recent one had 2 solutions.    


Canvas(sync_image_data=True, width=1050)